In [1]:
#optunaサンプル．以下サイト
#https://zenn.dev/merosu_radio/articles/2da39a25f91f05


# ライブラリ
import numpy as np
import pandas as pd
import japanize_matplotlib
import matplotlib.pyplot as plt
import optuna
import plotly

from pymoo.problems import get_problem
from pymoo.visualization.fitness_landscape import FitnessLandscape


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/yusaku/opt/anaconda3/envs/bo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def loop(ini_sample_np_seed,ini_sample_num,target_y,trial_max_num,sampler_seed,loop_num):
      
    ini_sample_np_seed = ini_sample_np_seed
    ini_sample_num = ini_sample_num
    target_y = target_y
    trial_max_num = trial_max_num
    sampler_seed = sampler_seed
    loop_num = loop_num

    if ini_sample_np_seed != -1:
        np.random.seed(seed=ini_sample_np_seed)
    
    x1 = np.random.uniform(-3, 3, ini_sample_num) 
    x2 = np.random.uniform(-3, 3, ini_sample_num)

    problem = get_problem("ackley", n_var=2, a=20, b=1/5, c=2 * np.pi)
    y=problem.evaluate(np.random.rand(ini_sample_num, 2)*3).flatten()

    df = pd.DataFrame({
        'x1': x1,
        'x2': x2,
        'y': y
    })

    if sampler_seed == -1:
        sampler = optuna.samplers.TPESampler(multivariate=True, n_startup_trials=0)
    else:
        sampler = optuna.samplers.TPESampler(multivariate=True, n_startup_trials=0,seed=sampler_seed)

    study = optuna.create_study(directions=['minimize'], sampler = sampler)

    # 探索範囲を指定
    search_space = {
        'x1': optuna.distributions.FloatDistribution(-3, 3),
        'x2': optuna.distributions.FloatDistribution(-3, 3),
    }

    # 実施した試験結果を追加
    for i, row in df.iterrows():

        params = {
            'x1':row['x1']
            ,'x2':row['x2']
            }   
        
        study.add_trial(optuna.trial.create_trial(
            params=params,
            distributions=search_space,
        
        # 目的変数はそれぞれ、目標値までの差分とする
            values=[row['y']-target_y]
        ))

    # 初期サンプルで目標を達成していたら終了する
    if min(study.trials_dataframe()['value']) < target_y:
        df_study = study.trials_dataframe()
        df_study['test']=''
        df_study['test'].loc[df_study['number'] <ini_sample_num]='ini_sample'
        df_study['test'].loc[df_study['number']>=ini_sample_num]='trial'
        trial_num=df_study.shape[0]-ini_sample_num
        df_info = pd.DataFrame({'test_func_liblary':'pymoo',
                            'test_func':'ackley',
                            'sampler':'TPSsampler',
                            'ini_smple_np_seed':[ini_sample_np_seed],
                            'ini_sample_num': [ini_sample_num],
                            'sampler_seed':[sampler_seed],
                            'trial_num': [trial_num],
                            'loop_num' : [loop_num]
                              }
                             )
        return(df_info)
    else:
        for i in range(trial_max_num):

            # Optunaで次の実験をレコメンド
            trial = study.ask()
            
            # 次の実験範囲を指定し、レコメンドした値をner_paramsに格納
            new_params = {}
            new_params['x1'] = trial.suggest_float('x1',-3, 3)
            new_params['x2'] = trial.suggest_float('x2',-3, 3)
            
            # 実験を行ったという設定で硬度、靭性を取得    
            new_y = problem.evaluate(np.array([new_params['x1'],new_params['x2']]))
            new_y = new_y
            
            # studyに実験結果を記録
            study.tell(trial, [new_y])
            # 目標値を上回れば終了
            if new_y <= target_y:
                break
            
        df_study = study.trials_dataframe()
        df_study['test']=''
        df_study['test'].loc[df_study['number'] <ini_sample_num]='ini_sample'
        df_study['test'].loc[df_study['number']>=ini_sample_num]='trial'

    trial_num=df_study.shape[0]-ini_sample_num

    '''
    path='./data/'
    file_name='test_func_liblary_pymoo_test_func_ackley_TPSsampler_ini_sample_np_seed_'\
    +str(ini_sample_np_seed)+'_'\
    +'ini_sample_num_'+str(ini_sample_num)+'_'\
    +'sampler_seed_'+str(sampler_seed)+'_'\
    +'trial_num_'+str(trial_num)+'_'\
    +'loop_num_'+str(loop_num)\
    +'.csv'
    '''

    df_info = pd.DataFrame({'test_func_liblary':'pymoo',
                            'test_func':'ackley',
                            'sampler':'TPSsampler',
                            'ini_smple_np_seed':[ini_sample_np_seed],
                            'ini_sample_num': [ini_sample_num],
                            'sampler_seed':[sampler_seed],
                            'trial_num': [trial_num],
                            'loop_num' : [loop_num]
                          }
                         )
    return(df_info)

    #df_info.to_csv(path+file_name,index=False)


In [3]:
'''
INI_SAMPLE_NUM = 1   #初期サンプル数
TARGET_Y = 1       #目標値
TRIAL_MAX_NUM = 100      #実験回数の上限値
SAMPLER_SEED = 10
'''

'\nINI_SAMPLE_NUM = 1   #初期サンプル数\nTARGET_Y = 1       #目標値\nTRIAL_MAX_NUM = 100      #実験回数の上限値\nSAMPLER_SEED = 10\n'

In [7]:
for num in range(100):
    #print(loop_num)
    #loop(ini_sample_np_seed,ini_sample_num,target_y,trial_max_num,sampler_seed,loop_num):
    df_tmp = loop(-1,100,1,100,-1,num)
    
    if num == 0:
        df_info = df_tmp
    else:
        df_info =pd.concat([df_info ,df_tmp],axis = 0) 

/Users/yusaku/opt/anaconda3/envs/bo/lib/python3.11/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-04-21 20:09:46,269] A new study created in memory with name: no-name-3fb8b602-43ce-44e2-920d-fe904bd72a8e
/var/folders/vg/t6pnts894zz63zk4_526d3k40000gn/T/ipykernel_53062/140557780.py:58: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to 

In [5]:
display(study.best_params)
display(study.best_value)
display(study.best_trials)

NameError: name 'study' is not defined

In [ ]:
fig = optuna.visualization.plot_slice(study, params=["x1", "x2"])
#fig.write_image('test.png')
fig = optuna.visualization.plot_optimization_history(study)
fig = optuna.visualization.plot_contour(study, params=["x1", "x2"])

In [ ]:
df_info.to_csv(path+file_name,index=False)

In [ ]:
#https://zenn.dev/msykoba/articles/791f8bed85496d
#サンプラーにはTPESampler（ベイズ最適化），NSGAIISampler（進化計算），BoTorchSampler（ベイズ最適化），QMCSampler（準モンテカルロ法），RandomSampler（ランダムサーチ），GridSampler（グリッドサーチ）